# 大模型

## IO：基于标准输入输出的流式调用

In [1]:
from textlong.io import TextBlock, stream_log

In [2]:
def hello():
    yield TextBlock("info", "hello textlong")
    yield TextBlock("chunk", "Textlong")
    yield TextBlock("chunk", "很")
    yield TextBlock("chunk", "棒！")

stream_log(hello)

>-[INFO]>> hello textlong
Textlong很棒！



{'output': 'Textlong很棒！', 'tools_call': {}}

## Tools Calling

仍然采纳`langchain`来定义工具，这可以大大简化回调工具的管理。

涉及定义工具的技巧主要包括：
- 使用 tool 装饰函数将新函数定义为工具类型
- 使用 StructTool 转换已有的函数为工具类型
- 使用 BaseModel 补充工具的参数定义

采用以上方法定义好工具，就可以使用 convert_to_openai_tool 将其转换为官方要求的 JSON 结构（确实会比手写节省很多精力）。

In [1]:
from textlong.llm.tools import tool, convert_to_openai_tool
from textlong.llm import qwen
from textlong.io import stream_log
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

stream_log(qwen, "今天广州天气如何啊", tools=[convert_to_openai_tool(get_current_weather)])

>-[TOOLS_CALL]>> {"index": 0, "id": "call_4ac210cf3b234d82840a89", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
>-[TOOLS_CALL]>> {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
>-[TOOLS_CALL]>> {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
>-[TOOLS_CALL]>> {"index": 0, "id": "", "type": "function", "function": {}}
{"0": {"index": 0, "id": "call_4ac210cf3b234d82840a89", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}


{'output': '',
 'tools_call': {0: {'index': 0,
   'id': 'call_4ac210cf3b234d82840a89',
   'type': 'function',
   'function': {'name': 'get_current_weather',
    'arguments': '{"location": "广州"}'}}}}

<div class="alert alert-info">
⚠️ 上面的调用仅仅是返回了工具描述，尚未真正执行工具代码。
</div>

## 模型支持

### OpenAI

In [2]:
from textlong.llm import openai
from textlong.io import stream_log

stream_log(
    openai,
    [{"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}],
    model="gpt-3.5-turbo"
)

小兔子在梦里
跳来跳去好开心
甜甜的胡萝卜味
梦里的世界美丽又甜



{'output': '小兔子在梦里\n跳来跳去好开心\n甜甜的胡萝卜味\n梦里的世界美丽又甜', 'tools_call': {}}

In [1]:
from textlong.llm.tools import tool, convert_to_openai_tool
from textlong.llm import openai
from textlong.io import stream_log
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

stream_log(openai, "今天广州天气如何啊", tools=[convert_to_openai_tool(get_current_weather)], model="gpt-3.5-turbo")

>-[TOOLS_CALL]>> {"index": 0, "function": {"id": "call_pHjBYW05KyMyzYjURI5g4FHr", "type": "function", "name": "get_current_weather", "arguments": ""}}
>-[TOOLS_CALL]>> {"index": 0, "function": {"id": "", "type": "function", "name": "", "arguments": "{\""}}
>-[TOOLS_CALL]>> {"index": 0, "function": {"id": "", "type": "function", "name": "", "arguments": "location"}}
>-[TOOLS_CALL]>> {"index": 0, "function": {"id": "", "type": "function", "name": "", "arguments": "\":\""}}
>-[TOOLS_CALL]>> {"index": 0, "function": {"id": "", "type": "function", "name": "", "arguments": "gu"}}
>-[TOOLS_CALL]>> {"index": 0, "function": {"id": "", "type": "function", "name": "", "arguments": "ang"}}
>-[TOOLS_CALL]>> {"index": 0, "function": {"id": "", "type": "function", "name": "", "arguments": "zhou"}}
>-[TOOLS_CALL]>> {"index": 0, "function": {"id": "", "type": "function", "name": "", "arguments": "\"}"}}
{"0": {"index": 0, "function": {"id": "call_pHjBYW05KyMyzYjURI5g4FHr", "type": "function", "name": "

{'output': '',
 'tools_call': {0: {'index': 0,
   'function': {'id': 'call_pHjBYW05KyMyzYjURI5g4FHr',
    'type': 'function',
    'name': 'get_current_weather',
    'arguments': '{"location":"guangzhou"}'}}}}

### 智谱AI

In [1]:
from textlong.llm import zhipu
from textlong.io import stream_log

stream_log(
    zhipu,
    [{"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}],
    model="glm-4-flash"
)

小兔子，梦飞扬，
夜幕降临入梦乡，
草地上，跳跃欢，
兔子王国梦工厂。



{'output': '小兔子，梦飞扬，\n夜幕降临入梦乡，\n草地上，跳跃欢，\n兔子王国梦工厂。', 'tools_call': {}}

In [4]:
from textlong.llm.tools import tool, convert_to_openai_tool
from textlong.llm import zhipu
from textlong.io import stream_log
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

stream_log(zhipu, "今天广州天气如何啊", tools=[convert_to_openai_tool(get_current_weather)], model="glm-4-air")

>-[TOOLS_CALL]>> {"index": 0, "function": {"id": "call_20240907213847cd86e6447ada47c0", "type": "function", "name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}
{"0": {"index": 0, "function": {"id": "call_20240907213847cd86e6447ada47c0", "type": "function", "name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}


{'output': '',
 'tools_call': {0: {'index': 0,
   'function': {'id': 'call_20240907213847cd86e6447ada47c0',
    'type': 'function',
    'name': 'get_current_weather',
    'arguments': '{"location": "广州"}'}}}}

### 通义千问

In [3]:
from textlong.llm import qwen
from textlong.io import stream_log

stream_log(
    qwen,
    [{"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}],
    model="qwen2-1.5b-instruct"
)

小白兔，梦中跑，  
胡萝卜，满天空飘。  
月亮船，带它翱翔，  
醒来笑，原来是梦乡。



{'output': '小白兔，梦中跑，  \n胡萝卜，满天空飘。  \n月亮船，带它翱翔，  \n醒来笑，原来是梦乡。',
 'tools_call': {}}

In [1]:
from textlong.llm.tools import tool, convert_to_openai_tool
from textlong.llm import qwen
from textlong.io import stream_log
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

stream_log(qwen, "今天广州天气如何啊", tools=[convert_to_openai_tool(get_current_weather)])

>-[TOOLS_CALL]>> {"index": 0, "id": "call_4ac210cf3b234d82840a89", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
>-[TOOLS_CALL]>> {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
>-[TOOLS_CALL]>> {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
>-[TOOLS_CALL]>> {"index": 0, "id": "", "type": "function", "function": {}}
{"0": {"index": 0, "id": "call_4ac210cf3b234d82840a89", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}


{'output': '',
 'tools_call': {0: {'index': 0,
   'id': 'call_4ac210cf3b234d82840a89',
   'type': 'function',
   'function': {'name': 'get_current_weather',
    'arguments': '{"location": "广州"}'}}}}

## Fake

### fake_llm: 模拟大模型调用过程

这可以观察提示语等内部结构。

In [4]:
from textlong.llm import fake_llm
from textlong.io import stream_log

stream_log(
    fake_llm,
    [
        {"role": "system", "content": "你是一个写作能手。"},
        {"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}
    ],
    model="qwen2-1.5b-instruct"
)

>-[INFO]>> system: 你是一个写作能手。
>-[INFO]>> user: 你能帮我写一首关于兔子做梦的四句儿歌?
这是一个模拟调用!



{'output': '这是一个模拟调用!', 'tools_call': {}}